In [1]:
import pandas as pd

import warnings
warnings.filterwarnings('ignore')

## Load data & resources

In [2]:
# # ISO 639 table for full language names
# iso_639_3_names = pd.read_csv("data/iso-639-3_Name_Index.tab.txt", delimiter="\t", index_col=0)
# iso_639_3_names = iso_639_3_names["Print_Name"].to_dict()
# iso_639_3_names["cmn"] = "Chinese"
# # iso_639_3_names

In [3]:
# # sentences for evaluating predicted language
# df = pd.read_csv("data/sentences.csv", delimiter="\t", header=None, index_col=0, names=["Language", "Sentence"])

## Get counts of sentences by language

In [4]:
# language_counts = df["Language"].value_counts(normalize=False)
# language_counts = language_counts.to_frame().reset_index()
# language_counts.rename(columns={"index":"ISO Name", "Language": "Count"}, inplace=True)
# language_counts["Full Name"] = language_counts["ISO Name"].map(iso_639_3_names)
# language_counts = language_counts[["ISO Name", "Full Name", "Count"]]

## Identify languages with > 5,000 sentences

In [5]:
# over_5k = language_counts[language_counts["Count"]>5000]
# over_5k_list = over_5k["ISO Name"].tolist()
# # over_5k_list # list of "ISO Name" str

## Check which common PKI languages are included in sentences

In [6]:
# PKI_LANGUAGES = [
#     "English",
#     "Chinese",
#     "German",
#     "Spanish",
#     "French",
#     "Italian",
#     "Japanese",
#     "Korean",
#     "Portuguese",
#     "Danish",
#     "Dutch", # no Dutch in sentences?
#     "Norwegian", # no Norwegian in sentences?
# ]

# found_PKI_LANGUAGES = over_5k[over_5k["Full Name"].isin(PKI_LANGUAGES)]
# # found_PKI_LANGUAGES

# set(PKI_LANGUAGES).difference(set(found_PKI_LANGUAGES["Full Name"]))

> sentences are missing Dutch & Norwegian.

## Limit to languages with over 5,000 sentences & that are common at PKI

In [7]:
# sentences_over_5k = df[df["Language"].isin(over_5k_list)]
# sentences_over_5k["Language Full Name"] = sentences_over_5k["Language"].map(iso_639_3_names)
# sentences_over_5k = sentences_over_5k[sentences_over_5k["Language Full Name"].isin(PKI_LANGUAGES)]
# sentences_over_5k = sentences_over_5k[["Sentence", "Language", "Language Full Name"]]

## Create a random stratified sample of 10,000 sentences

In [8]:
# max_sample_size = int(sentences_over_5k["Language Full Name"].value_counts(normalize=False).min() / 1000) * 1000 # use the smallest language group to set uniform sample size across all languages
# # max_sample_size

In [9]:
# sample = sentences_over_5k.groupby("Language Full Name", group_keys=False).apply(lambda x: x.sample(max_sample_size))

In [10]:
from create_Tatoeba_train_test import get_language_mappings, get_sentence_word_char_len

In [11]:
language_mappings = get_language_mappings("language_mappings.csv")
iso_639_3_English = language_mappings[["ISO 639-3", "English Name"]].set_index(keys="ISO 639-3")["English Name"].to_dict()
# iso_639_3_English

## Load sample train data set

In [12]:
sample = pd.read_csv("output/Tatoeba_stratify_train_2021-7-29_1520.csv")
sample["Language_English_Name"] = sample["Language"].map(iso_639_3_English)
sample["Sentence_len"] = sample.apply(lambda row: get_sentence_word_char_len(row), axis=1)

In [13]:
bin_labels = ["1","2","3","4","5","6","7","8","9","10","11 to 16", "17 to 27", "28 to 48", "49 to 99"]

sample["Sentence_len_bin"] = pd.cut(
    x=sample["Sentence_len"],
    bins=[0,1,2,3,4,5,6,7,8,9,10,16,27,48,99],
    labels=bin_labels
)

In [14]:
sample.sample(10)

,Original Index,Sentence,Language,Language_English_Name,Sentence_len,Sentence_len_bin
32919,8368313,톰한테 말해.,kor,Korean,5,5
12903,1296257,"Han var ikke så træt, som han så ud til at være.",dan,Danish,12,11 to 16
24400,5174459,Har du lyst til at tage med mig ud at handle?,dan,Danish,11,11 to 16
30321,1372961,Je pense qu'il est temps pour moi de consulter...,fra,French,11,11 to 16
18435,4865831,Tom nem sequer olhou para o relatório que Mari...,por,Portuguese,11,11 to 16
32214,8363670,"다음 날, 눈사람은 완전히 녹아버렸어.",kor,Korean,16,11 to 16
17410,527939,他整天都在埋頭埋腦地看小說，連飯也沒有吃。,cmn,Chinese,19,17 to 27
35972,7505057,I thought Tom would like to be the next one to...,eng,English,13,11 to 16
61102,6197277,"Hvor er hagen ved det? ""Der er ingen. Det svær...",dan,Danish,11,11 to 16
56903,2487114,Eu não brigo com pessoas.,por,Portuguese,5,5


In [15]:
round(sample["Language_English_Name"].value_counts(normalize=True) * 100,2)

Dutch         9.09
Spanish       9.09
Portuguese    9.09
Italian       9.09
German        9.09
Chinese       9.09
English       9.09
Korean        9.09
Danish        9.09
French        9.09
Japanese      9.09
Name: Language_English_Name, dtype: float64

# langid

#### Setup for langid

In [16]:
from langid.langid import LanguageIdentifier, model

In [17]:
iso_639_3_iso_639_1 = language_mappings[["ISO 639-3", "ISO 639-1"]].set_index(keys="ISO 639-3")["ISO 639-1"].to_dict()
# iso_639_3_iso_639_1

In [18]:
# find all languages to include in langid predictions
set_languages = [iso_639_3_iso_639_1[i] for i in sample["Language"].unique().tolist()]

lang_identifier = LanguageIdentifier.from_modelstring(model, norm_probs=True)
# lang_identifier.set_languages(set_languages)

> if we cannot limit all language prediction libraries then don't limit any (?)

#### Generate langid predictions

In [19]:
%%time
# 14s, with limited languages
# 2min, 45s, with no limits on languages

sample["langid"] = sample.apply(lambda row: list(lang_identifier.classify(row["Sentence"])), axis=1)

Wall time: 2min 45s


In [20]:
sample["langid_language"], sample["langid_score"] = sample["langid"].str
sample.drop(columns=["langid"], inplace=True)

In [21]:
# map Language to English Name & langid_language to English Name
ISO_639_3 = language_mappings.set_index("ISO 639-3")
ISO_639_3 = ISO_639_3["English Name"].to_dict()
sample["Language_English_Name"] = sample["Language"].map(ISO_639_3)

ISO_639_1 = language_mappings.set_index("ISO 639-1")
ISO_639_1 = ISO_639_1["English Name"].to_dict()
sample["langid_language_English_Name"] = sample["langid_language"].map(ISO_639_1)

In [31]:
# round(sample["langid_language_English_Name"].value_counts(normalize=True, dropna=False) * 100,2)
round(sample["langid_language"].value_counts(normalize=True, dropna=False) * 100,2)

ja    9.48
en    9.22
fr    9.14
ko    9.09
de    9.06
it    8.96
nl    8.91
es    8.72
zh    8.70
pt    8.48
da    6.23
no    1.76
nb    0.90
gl    0.49
ca    0.17
nn    0.09
af    0.05
sv    0.05
eo    0.04
br    0.04
hu    0.03
ro    0.03
ku    0.02
fi    0.02
sl    0.02
oc    0.02
an    0.02
et    0.02
eu    0.02
sk    0.02
cs    0.02
lt    0.01
hr    0.01
lb    0.01
ga    0.01
id    0.01
wa    0.01
mg    0.01
jv    0.01
tr    0.01
ht    0.01
pl    0.01
mt    0.01
bs    0.01
is    0.01
cy    0.00
lv    0.00
tl    0.00
sq    0.00
la    0.00
xh    0.00
qu    0.00
se    0.00
fo    0.00
ky    0.00
Name: langid_language, dtype: float64

# langdetect

In [23]:
from langdetect import DetectorFactory
DetectorFactory.seed = 7

In [24]:
from langdetect import detect_langs

In [25]:
%%time
# 3min 34s

sample["langdetect"] = sample.apply(lambda row: detect_langs(row["Sentence"])[0], axis=1)

Wall time: 3min 22s


In [26]:
sample["langdetect_language"] = sample.apply(lambda row: row["langdetect"].lang, axis=1)
sample["langdetect_score"] = sample.apply(lambda row: row["langdetect"].prob, axis=1)
sample.drop(columns=["langdetect"], inplace=True)

In [27]:
sample.sample(10)

,Original Index,Sentence,Language,Language_English_Name,Sentence_len,Sentence_len_bin,langid_language,langid_score,langid_language_English_Name,langdetect_language,langdetect_score
12419,7093267,É a primeira vez que visito o Rio de Janeiro.,por,Portuguese,10,10,pt,0.948547,Portuguese,pt,0.999997
39184,3210186,なんでお前ここにいんの？,jpn,Japanese,11,11 to 16,ja,1.000000,Japanese,ja,1.000000
34960,5861714,In der Wohnung oben steigt gerade eine Party.,deu,German,8,8,de,1.000000,German,de,0.999996
31715,2006393,I wish I had a tackle box as nice as yours.,eng,English,11,11 to 16,en,0.999562,English,en,0.999997
43733,1731968,我真的很想知道他為甚麼會做出這種事來。,cmn,Chinese,18,17 to 27,zh,1.000000,Chinese,zh-tw,0.571429
33994,10156330,Das rassistische und kolonialistische System d...,deu,German,16,11 to 16,de,1.000000,German,de,0.999997
28740,5155527,Mindst en gang om ugen vasker han sin bil.,dan,Danish,9,9,nl,0.834431,Dutch,da,0.999992
24391,211844,その会社の収益性が改善した。,jpn,Japanese,13,11 to 16,ja,1.000000,Japanese,ja,1.000000
46730,4749595,메리가 피를 흘리고 있어.,kor,Korean,10,10,ko,1.000000,Korean,ko,0.999999
1327,8622555,Hylderne med toiletpapir var fuldstændig tomme...,dan,Danish,17,17 to 27,da,0.992456,Danish,da,0.999994


In [28]:
raise Exception("continue here: map langdetect_language to an English name")

Exception: continue here: map langdetect_language to an English name

In [29]:
round(sample["langdetect_language"].value_counts(normalize=True, dropna=False) * 100,2)

ko       11.78
fr        9.12
pt        9.07
de        9.04
ja        9.02
en        8.96
it        8.93
es        8.59
nl        8.31
da        7.45
zh-cn     5.06
no        1.60
zh-tw     1.30
af        0.84
ca        0.36
ro        0.10
sv        0.08
tl        0.04
cy        0.04
id        0.04
et        0.04
sl        0.03
sk        0.03
so        0.02
lt        0.02
hu        0.02
fi        0.02
hr        0.02
sw        0.01
vi        0.01
pl        0.01
tr        0.01
cs        0.00
sq        0.00
lv        0.00
Name: langdetect_language, dtype: float64

# textblob

In [ ]:
raise Exception("pause here")

## Evaluate overall results

In [ ]:
pd.crosstab(sample["Language_English_Name"], sample["langid_language_English_Name"])

In [ ]:
# micro-F1 for langid & others

In [ ]:
from sklearn.metrics import classification_report

In [ ]:
# labels = sample["Language_English_Name"].unique().tolist()
# # labels

In [ ]:
# print(classification_report(sample["Language_English_Name"], sample["langid_language_English_Name"], target_names=labels))

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import itertools

In [ ]:
def plot_confusion_matrix(cm,
                          target_names,
                          title='Confusion matrix',
                          cmap=None,
                          normalize=True):
    """
    given a sklearn confusion matrix (cm), make a nice plot

    Arguments
    ---------
    cm:           confusion matrix from sklearn.metrics.confusion_matrix

    target_names: given classification classes such as [0, 1, 2]
                  the class names, for example: ['high', 'medium', 'low']

    title:        the text to display at the top of the matrix

    cmap:         the gradient of the values displayed from matplotlib.pyplot.cm
                  see http://matplotlib.org/examples/color/colormaps_reference.html
                  plt.get_cmap('jet') or plt.cm.Blues

    normalize:    If False, plot the raw numbers
                  If True, plot the proportions

    Usage
    -----
    plot_confusion_matrix(cm           = cm,                  # confusion matrix created by
                                                              # sklearn.metrics.confusion_matrix
                          normalize    = True,                # show proportions
                          target_names = y_labels_vals,       # list of names of the classes
                          title        = best_estimator_name) # title of graph

    Citiation
    ---------
    http://scikit-learn.org/stable/auto_examples/model_selection/plot_confusion_matrix.html

    """
    
    accuracy = np.trace(cm) / np.sum(cm).astype('float')
    misclass = 1 - accuracy

    if cmap is None:
        cmap = plt.get_cmap('Purples')

    plt.figure(figsize=(20, 10))
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()

    if target_names is not None:
        tick_marks = np.arange(len(target_names))
        plt.xticks(tick_marks, target_names, rotation=90)
        plt.yticks(tick_marks, target_names)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]


    thresh = cm.max() / 1.5 if normalize else cm.max() / 2
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        if normalize:
            plt.text(j, i, "{:0.4f}".format(cm[i, j]),
                     horizontalalignment="center",
                     color="white" if cm[i, j] > thresh else "black")
        else:
            plt.text(j, i, "{:,}".format(cm[i, j]),
                     horizontalalignment="center",
                     color="white" if cm[i, j] > thresh else "black")


    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.show()

In [ ]:
from sklearn.metrics import confusion_matrix

In [ ]:
# cm = confusion_matrix(sample["Language_English_Name"], sample["langid_language_English_Name"], labels)
# plot_confusion_matrix(cm=cm, target_names=labels, normalize=False)

## Evaluate results as function of sentence length

In [ ]:
from sklearn.metrics import f1_score

In [ ]:
results = []
for this_bin in bin_labels:
    print(this_bin)
    
#     for i in range(0,100):        
#         temp_sample = sample[sample["bin"]==this_bin]
#         # find max sample size
#         max_sample_size = temp_sample["Language Full Name"].value_counts(normalize=False).min()
#         this_sample = temp_sample.groupby("Language Full Name", group_keys=False).apply(lambda x: x.sample(max_sample_size))

#         this_sample["langid"] = this_sample.apply(lambda row: list(lang_identifier.classify(row["Sentence"])), axis=1)
#         this_sample["langid_language"], this_sample["langid_score"] = this_sample["langid"].str
#         this_sample.drop(columns=["langid"], inplace=True)
#         this_sample["langid_language Full Name"] = this_sample["langid_language"].map(iso_639_1_names)

#         these_labels = this_sample["Language Full Name"].unique().tolist()

#         for lang in this_sample["langid_language Full Name"].unique():
#             these_results = this_sample[this_sample["langid_language Full Name"]==lang]

#             this_f1 = f1_score(these_results["Language Full Name"], these_results["langid_language Full Name"], labels=these_labels, average="weighted")
#             result = (this_bin, lang, round(this_f1,2))
#             results.append(result)

### Plot results

In [ ]:
import seaborn as sns

In [ ]:
# TODO: address inconsistent sample sizes (?)

In [ ]:
results = pd.DataFrame(data=results, columns=["Bin", "Language", "F1"])
results_pivot = results.pivot_table(values="F1", index="Language", columns="Bin")
results_pivot = results_pivot[bin_labels]
results_pivot.loc["Average"] = results_pivot.mean()

In [ ]:
# create custom colormap for consistent colors
language_cmap = {}
language_cmap["Average"] = (0.8901960784313725, 0.10196078431372549, 0.10980392156862745) # red-orange

language_cmap["Chinese"] = (0.5545098039215688, 0.7564167627835449, 0.8683121876201461) # blue
language_cmap["Japanese"] = (0.21568627450980393, 0.5294117647058824, 0.7542483660130719) # blue
language_cmap["Korean"] = (0.06251441753171857, 0.35750865051903113, 0.6429065743944637) # blue

language_cmap["French"] = (0.5739331026528259, 0.8241753171856978, 0.5606151480199923) # green
language_cmap["Italian"] = (0.21568627450980393, 0.6287581699346405, 0.3333333333333333) # green
language_cmap["Spanish"] = (0.04359861591695502, 0.4648212226066897, 0.20369088811995384) # green
language_cmap["Portuguese"] = (0.7792233756247597, 0.9132333717800846, 0.7518031526336024) # green

language_cmap["English"] = (0.698961937716263, 0.6965013456362938, 0.8372164552095348) # purple
language_cmap["Danish"] = (0.47320261437908495, 0.43267973856209146, 0.6993464052287581) # purple
language_cmap["German"] = (0.3568166089965398, 0.20525951557093425, 0.5856978085351787) # purple

# create custom dash dict for consistent dashes
language_dashes = {}
language_dashes["Average"] = (1, 1)

language_dashes["Chinese"] = (2, 1)
language_dashes["Japanese"] = (4, 3)
language_dashes["Korean"] = (6, 6)

language_dashes["French"] = (2, 1)
language_dashes["Italian"] = (4, 3)
language_dashes["Spanish"] = (6, 6)
language_dashes["Portuguese"] = (8, 12)

language_dashes["English"] = (2, 1)
language_dashes["Danish"] = (4, 3)
language_dashes["German"] = (6, 6)

In [ ]:
# ax = results_pivot.T.plot(figsize=(20,10), title="Average F1 Score By Sentence Character Length", ylabel="F1 Score", xlabel="Sentence Character Length", ylim=(0.0,1.1));
# ax.axhline(y=0.8, linestyle="--", color="gray", alpha=0.3)
# ax.set_xticks(range(len(bin_labels)));
# ax.set_xticklabels(bin_labels);
plt.figure(figsize=(30,10))
plt.title("Average F1 Score By Sentence Length")
plt.xlabel("Sentence Length")
plt.ylabel("F1 Score")
plt.ylim(0.0, 1.1)
plt.axhline(y=0.8, linestyle="--", color="gray", alpha=0.3);
sns.lineplot(data=results_pivot.T, palette=language_cmap, dashes=language_dashes);
plt.legend(loc="lower right");

In [ ]:
# ax = results_pivot.loc["Average"].T.plot(figsize=(20,10), title="Average F1 Score By Sentence Character Length", ylabel="F1 Score", xlabel="Sentence Character Length", ylim=(0.0,1.1));
# ax.axhline(y=0.8, linestyle="--", color="gray", alpha=0.3)
# ax.set_xticks(range(len(bin_labels)));
# ax.set_xticklabels(bin_labels);
plt.figure(figsize=(30,10))
plt.title("Average F1 Score By Sentence Length")
plt.xlabel("Sentence Length")
plt.ylabel("F1 Score")
plt.ylim(0.0, 1.1)
plt.axhline(y=0.8, linestyle="--", color="gray", alpha=0.3);
sns.lineplot(data=results_pivot.loc["Average"].T, color=language_cmap["Average"], linestyle="dotted");
# plt.legend(loc="lower right");

In [ ]:
THESE_LANGUAGES = ["Chinese", "Japanese", "Korean"]
# ax = results_pivot.loc[THESE_LANGUAGES+["Average"]].T.plot(figsize=(20,10), title="Average F1 Score By Sentence Character Length", ylabel="F1 Score", xlabel="Sentence Character Length", ylim=(0.0,1.1));
# ax.axhline(y=0.8, linestyle="--", color="gray", alpha=0.3)
# ax.set_xticks(range(len(bin_labels)));
# ax.set_xticklabels(bin_labels);
plt.figure(figsize=(30,10))
plt.title("Average F1 Score By Sentence Length")
plt.xlabel("Sentence Length")
plt.ylabel("F1 Score")
plt.ylim(0.0, 1.1)
plt.axhline(y=0.8, linestyle="--", color="gray", alpha=0.3);
sns.lineplot(data=results_pivot.loc[THESE_LANGUAGES+["Average"]].T, palette=language_cmap, dashes=language_dashes);
plt.legend(loc="lower right");

In [ ]:
THESE_LANGUAGES = ['Danish', 'English', 'French', 'German', 'Italian', 'Portuguese', 'Spanish']
# ax = results_pivot.loc[THESE_LANGUAGES+["Average"]].T.plot(figsize=(20,10), title="Average F1 Score By Sentence Character Length", ylabel="F1 Score", xlabel="Sentence Character Length", ylim=(0.0,1.1));
# ax.axhline(y=0.8, linestyle="--", color="gray", alpha=0.3)
# ax.set_xticks(range(len(bin_labels)));
# ax.set_xticklabels(bin_labels);
plt.figure(figsize=(30,10))
plt.title("Average F1 Score By Sentence Length")
plt.xlabel("Sentence Length")
plt.ylabel("F1 Score")
plt.ylim(0.0, 1.1)
plt.axhline(y=0.8, linestyle="--", color="gray", alpha=0.3);
sns.lineplot(data=results_pivot.loc[THESE_LANGUAGES+["Average"]].T, palette=language_cmap, dashes=language_dashes);
plt.legend(loc="lower right");

In [ ]:
THESE_LANGUAGES = ['Danish', 'English', 'German']
# ax = results_pivot.loc[THESE_LANGUAGES+["Average"]].T.plot(figsize=(20,10), title="Average F1 Score By Sentence Character Length", ylabel="F1 Score", xlabel="Sentence Character Length", ylim=(0.0,1.1));
# ax.axhline(y=0.8, linestyle="--", color="gray", alpha=0.3)
# ax.set_xticks(range(len(bin_labels)));
# ax.set_xticklabels(bin_labels);
plt.figure(figsize=(30,10))
plt.title("Average F1 Score By Sentence Length")
plt.xlabel("Sentence Length")
plt.ylabel("F1 Score")
plt.ylim(0.0, 1.1)
plt.axhline(y=0.8, linestyle="--", color="gray", alpha=0.3);
sns.lineplot(data=results_pivot.loc[THESE_LANGUAGES+["Average"]].T, palette=language_cmap, dashes=language_dashes);
plt.legend(loc="lower right");

In [ ]:
THESE_LANGUAGES = ['French', 'Italian', 'Portuguese', 'Spanish']
# ax = results_pivot.loc[THESE_LANGUAGES+["Average"]].T.plot(figsize=(20,10), title="Average F1 Score By Sentence Character Length", ylabel="F1 Score", xlabel="Sentence Character Length", ylim=(0.0,1.1));
# ax.axhline(y=0.8, linestyle="--", color="gray", alpha=0.3)
# ax.set_xticks(range(len(bin_labels)));
# ax.set_xticklabels(bin_labels);
plt.figure(figsize=(30,10))
plt.title("Average F1 Score By Sentence Length")
plt.xlabel("Sentence Length")
plt.ylabel("F1 Score")
plt.ylim(0.0, 1.1)
plt.axhline(y=0.8, linestyle="--", color="gray", alpha=0.3);
sns.lineplot(data=results_pivot.loc[THESE_LANGUAGES+["Average"]].T, palette=language_cmap, dashes=language_dashes);
plt.legend(loc="lower right");

In [ ]:
# number of chars needed to be above 0.8 F1

In [ ]:
results_pivot

In [ ]:
# find the first bin where average F1 exceeds 0.8 & stays above 0.8 for the remainder

In [ ]:
_80 = results_pivot.apply(lambda row: row[row > 0.79].index[0], axis=1)
_90 = results_pivot.apply(lambda row: row[row > 0.89].index[0], axis=1)
results_pivot["First Bin >= 0.8"] = _80
results_pivot["First Bin >= 0.9"] = _90
results_pivot

In [ ]:
results_pivot["First Bin >= 0.8"].sort_values()

In [ ]:
results_pivot["First Bin >= 0.9"].sort_values()

In [ ]:
# TODO: evaluate langdetect & textblob libs
# TODO: evaluate impact of lowercasing, removing punctuation, removing numbers